<a href="https://colab.research.google.com/github/CodeCraftIA/Options-Gamma-Exposure-Analysis-Tool/blob/main/gex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
import os
from datetime import timedelta, datetime
import json
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import pandas as pd
import requests
from matplotlib import dates

# Set plot style
plt.style.use("seaborn-dark")
for param in ["figure.facecolor", "axes.facecolor", "savefig.facecolor"]:
    plt.rcParams[param] = "#212946"
for param in ["text.color", "axes.labelcolor", "xtick.color", "ytick.color"]:
    plt.rcParams[param] = "0.9"

contract_size = 100


def run(ticker, expiration_date):
    # Convert expiration_date from mm/dd/yy to yyyy-mm-dd format for internal processing
    expiration_date = datetime.strptime(expiration_date, "%m/%d/%y").strftime("%Y-%m-%d")

    spot_price, option_data = scrape_data(ticker, expiration_date)
    compute_total_gex(spot_price, option_data)
    gex_by_strike_data = compute_gex_by_strike(spot_price, option_data)  # Ensure this function returns the DataFrame

    print("\n")  # Add a visual gap before the zero GEX level output
    calculate_zero_gex_level(gex_by_strike_data)  # Call the new function with the appropriate data
    print("\n")  # Optionally, add another visual gap after the output


def scrape_data(ticker, expiration_date):
    """Scrape data from CBOE website"""

    try:
        data = requests.get(
            f"https://cdn.cboe.com/api/global/delayed_quotes/options/_{ticker}.json"
        )
        with open(f"{ticker}.json", "w") as f:
            json.dump(data.json(), f)

    except ValueError:
        data = requests.get(
            f"https://cdn.cboe.com/api/global/delayed_quotes/options/{ticker}.json"
        )
        with open(f"{ticker}.json", "w") as f:
            json.dump(data.json(), f)
    # Convert json to pandas DataFrame
    data = pd.DataFrame.from_dict(data.json())

    spot_price = data.loc["current_price", "data"]
    option_data = pd.DataFrame(data.loc["options", "data"])

    # Fix and filter option data based on expiration date
    fixed_data = fix_option_data(option_data)
    filtered_data = filter_data_by_expiration(fixed_data, expiration_date)

    return spot_price, filtered_data

def filter_data_by_expiration(data, expiration_date):
    """
    Filter options data for a specific expiration date.
    """
    # Ensure expiration_date is in the correct format (assuming YYYY-MM-DD)
    data['expiration'] = pd.to_datetime(data['expiration'])
    desired_date = pd.to_datetime(expiration_date)

    # Filter data
    filtered_data = data[data['expiration'] == desired_date]

    return filtered_data

def fix_option_data(data):
    """
    Fix option data columns.

    From the name of the option derive type of option, expiration and strike price
    """
    data["type"] = data.option.str.extract(r"\d([A-Z])\d")
    data["strike"] = data.option.str.extract(r"\d[A-Z](\d+)\d\d\d").astype(int)
    data["expiration"] = data.option.str.extract(r"[A-Z](\d+)").astype(str)
    # Convert expiration to datetime format
    data["expiration"] = pd.to_datetime(data["expiration"], format="%y%m%d")

    return data


def compute_total_gex(spot, data):
    """Compute dealers' total GEX"""
    # Compute gamma exposure for each option
    data["GEX"] = spot * data.gamma * data.open_interest * contract_size * spot * 0.01

    # For put option we assume negative gamma, i.e. dealers sell puts and buy calls
    data["GEX"] = data.apply(lambda x: -x.GEX if x.type == "P" else x.GEX, axis=1)
    print(f"Total notional GEX: ${round(data.GEX.sum() / 10 ** 9, 4)} Bn")

def calculate_zero_gex_level(data):
    """Calculate the strike price with zero Gamma Exposure (GEX)."""
    # Assuming data is already filtered for relevant strikes and expirations
    # Sort data by strike
    data_sorted = data.sort_values("strike")

    # Initialize previous values
    prev_strike = prev_gex = None

    # Iterate over rows
    for index, row in data_sorted.iterrows():
        strike, gex = row['strike'], row['GEX']

        # Check if GEX crosses from positive to negative or vice versa
        if prev_gex is not None and ((prev_gex < 0 < gex) or (prev_gex > 0 > gex)):
            # Linear interpolation to find the zero crossing point
            zero_gex_strike = prev_strike + (0 - prev_gex) * (strike - prev_strike) / (gex - prev_gex)
            print(f"Zero GEX Level Estimated at Strike: {zero_gex_strike:.2f}")
            return

        prev_strike, prev_gex = strike, gex

    print("No zero GEX level found.")

'''
def compute_gex_by_strike(spot, data):
    """Compute and plot GEX by strike"""
    print("\n")  # Adds a visual gap before plotting
    # Compute total GEX by strike
    gex_by_strike = data.groupby("strike")["GEX"].sum() / 10**9

    # Limit data to +- 15% from spot price
    limit_criteria = (gex_by_strike.index > spot * 0.85) & (gex_by_strike.index < spot * 1.15)

    # Plot GEX by strike
    plt.bar(
        gex_by_strike.loc[limit_criteria].index,
        gex_by_strike.loc[limit_criteria],
        color="#FE53BB",
        alpha=0.5,
    )
    plt.grid(color="#2A3459")
    plt.xticks(fontweight="heavy")
    plt.yticks(fontweight="heavy")
    plt.xlabel("Strike", fontweight="heavy")
    plt.ylabel("Gamma Exposure (Bn$ / %)", fontweight="heavy")
    plt.title(f"{ticker} GEX by strike", fontweight="heavy")
    plt.show()
    return data.groupby("strike")["GEX"].sum().reset_index()
'''
'''
def compute_gex_by_strike(spot, data):
    """Compute and plot GEX by strike using Plotly"""
    print("\n")  # Adds a visual gap before plotting

    # Compute total GEX by strike
    gex_by_strike = data.groupby("strike")["GEX"].sum() / 10**9

    # Limit data to +- 15% from spot price
    limit_criteria = (gex_by_strike.index > spot * 0.85) & (gex_by_strike.index < spot * 1.15)
    filtered_gex_by_strike = gex_by_strike.loc[limit_criteria]

    # Create a figure
    fig = go.Figure(data=go.Bar(
        x=filtered_gex_by_strike.index,
        y=filtered_gex_by_strike,
        marker_color="#FE53BB",
        opacity=0.5
    ))

    # Customizing the layout to match the initial style preferences
    fig.update_layout(
        title=f"{ticker} GEX by Strike",
        xaxis_title="Strike",
        yaxis_title="Gamma Exposure (Bn$ / %)",
        plot_bgcolor="#212946",  # Background color
        paper_bgcolor="#212946",  # Overall background color, outside the plotting area
        font=dict(family="Courier New, monospace", size=18, color="#E0E0E0"),  # Text and title colors
        xaxis=dict(tickmode='linear', color="#E0E0E0"),  # X-axis tick color
        yaxis=dict(gridcolor="#2A3459", color="#E0E0E0"),  # Y-axis tick and grid color
    )

    # Update axes and gridline colors
    fig.update_xaxes(showline=True, linewidth=2, linecolor="#E0E0E0", gridcolor="#2A3459")
    fig.update_yaxes(showline=True, linewidth=2, linecolor="#E0E0E0", gridcolor="#2A3459")

    # Show figure
    fig.show()
    return data.groupby("strike")["GEX"].sum().reset_index()
'''
def compute_gex_by_strike(spot, data):
    """Compute and plot GEX by strike using Plotly with dynamic x-axis tick step."""
    print("\n")  # Adds a visual gap before plotting

    # Compute total GEX by strike
    gex_by_strike = data.groupby("strike")["GEX"].sum() / 10**9

    # Limit data to +- 15% from spot price
    limit_criteria = (gex_by_strike.index > spot * 0.85) & (gex_by_strike.index < spot * 1.15)
    filtered_gex_by_strike = gex_by_strike.loc[limit_criteria]

    # Determine the dynamic step size for x-axis ticks based on the range of strike prices
    strike_range = filtered_gex_by_strike.index.max() - filtered_gex_by_strike.index.min()
    num_ticks = 10  # Desired number of ticks - adjust as needed
    tick_step = round(strike_range / num_ticks, -1)  # Round to nearest 10 for simplicity; adjust logic as needed

    # Adjust tick_step to ensure it is not too small
    if tick_step < 2:
        tick_step = 2
    elif tick_step > 2 and tick_step < 2.5:
        tick_step = 2.5

    # Create a figure
    fig = go.Figure(data=go.Bar(
        x=filtered_gex_by_strike.index,
        y=filtered_gex_by_strike,
        marker_color="#FE53BB",
        opacity=0.5
    ))

    # Customizing the layout
    fig.update_layout(
        title=f"{ticker} GEX by Strike",
        xaxis_title="Strike",
        yaxis_title="Gamma Exposure (Bn$ / %)",
        plot_bgcolor="#212946",
        paper_bgcolor="#212946",
        font=dict(family="Courier New, monospace", size=18, color="#E0E0E0"),
        xaxis=dict(
            tickmode='linear',
            tick0=min(filtered_gex_by_strike.index),
            dtick=tick_step,
            color="#E0E0E0"
        ),
        yaxis=dict(gridcolor="#2A3459", color="#E0E0E0"),
        width=700
    )

    # Update axes and gridline colors
    fig.update_xaxes(showline=True, linewidth=2, linecolor="#E0E0E0", gridcolor="#2A3459")
    fig.update_yaxes(showline=True, linewidth=2, linecolor="#E0E0E0", gridcolor="#2A3459")

    fig.show()
    return data.groupby("strike")["GEX"].sum().reset_index()
if __name__ == "__main__":
    print("\n")  # Optional: Add an initial gap for better readability
    ticker = input("Enter desired ticker: ").upper()
    expiration_date = input("Enter expiration date (MM/DD/YY): ")  # Adjust the prompt to reflect the new format
    print("\n")  # Add a visual gap between input and the start of the process
    run(ticker, expiration_date)

<ipython-input-5-f06b5c3d8e44>:11: MatplotlibDeprecationWarning:

The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.





Enter desired ticker: TSLA
Enter expiration date (MM/DD/YY): 07/05/24


Total notional GEX: $0.2752 Bn






Zero GEX Level Estimated at Strike: 196.90


